In [27]:
import re
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,TemplateSendMessage,PostbackEvent,DatetimePickerTemplateAction 
)

from linebot.models.template import (
    ButtonsTemplate,PostbackTemplateAction,MessageTemplateAction,URITemplateAction
)
from pymongo import MongoClient
from linebot.models import (
    TemplateSendMessage, ButtonsTemplate,
    PostbackTemplateAction, MessageTemplateAction,
    URITemplateAction, DatetimePickerTemplateAction,
    ConfirmTemplate, CarouselTemplate, CarouselColumn,
    ImageCarouselTemplate, ImageCarouselColumn
)
class MongoConnection(object):
    client = None
    
    @staticmethod
    def getConnection():
        if MongoConnection.client is None:
            MongoConnection.client = MongoClient("mongodb://10.120.37.108:27017")
            return MongoConnection.client
        else:
            return MongoConnection.client
from datetime import datetime 
start = datetime.strptime("2017-10-01", "%Y-%m-%d")
end = datetime.strptime("2018-01-01", "%Y-%m-%d")
client = MongoConnection.getConnection()
db = client["project"]
collection_comments = db["comments"]
collection_keywords = db["keywords"]
collection_posts    = db["posts"]

In [28]:
politician_dict={
   "柯p":"136845026417486", 
   "柯市長":"136845026417486", 
   "柯文哲":"136845026417486", 
   "蔡英文":"46251501064", 
   "蔡總統":"46251501064",
   "李錫錕":"360151611020961", 
   "錕P":"360151611020961", 
   "黃國昌":"449664581882455", 
   "戰神":"449664581882455", 
   "侯友宜":"261813197541354",  
   "江啟臣":"191690867518507", 
   "陳其邁":"122936517768637", 
   "鄭文燦":"333058400178329", 
   "賴清德":"152472278103133", 
   "洪慈庸":"852926604746233", 
   "朱立倫":"10150145806225128", 
   "林昶佐":"365320250345879", 
   "蔣萬安":"805460986214082",
   "姚文智":"1380211668909443", 
   "蔡正元": "184799244894343"
}
politician_names = politician_dict.keys()

kmt_politician_dict={
   "朱立倫":"10150145806225128",
   "侯友宜":"261813197541354",
   "蔣萬安":"805460986214082",
   "蔡正元": "184799244894343",
   "江啟臣":"191690867518507"
}
kmt_politician_names = kmt_politician_dict.keys()

dpp_politician_dict={
   "蔡英文":"46251501064",
   "陳其邁":"122936517768637", 
   "鄭文燦":"333058400178329", 
   "賴清德":"152472278103133",
   "姚文智":"1380211668909443"
}
dpp_politician_names = dpp_politician_dict.keys()

power_politician_dict={
   "黃國昌":"449664581882455", 
   "洪慈庸":"852926604746233",
   "林昶佐":"365320250345879"
}
power_politician_names = power_politician_dict.keys()

noparty_politician_dict={
   "柯文哲":"136845026417486",  
   "李錫錕":"360151611020961",
}
noparty_politician_names = noparty_politician_dict.keys()

In [29]:
def data(event):
    data_list=[]
    for politician_name in politician_names:
        if len(re.findall(politician_name, event)) !=0 :
            
            ####hot
            hot = collection_comments.count({"politician_id":politician_dict[politician_name], 
                                             "created_time": {"$gte": start, "$lte": end}})
            
            ####ten_keywords
            keywords_list = collection_keywords.find({"politician_id":politician_dict[politician_name], 
                                                      "created_time": {"$gte": start, "$lte": end}},
                                         {"_id":0,"word":1,"link":1}).sort([("new_tfidf", -1)]).limit(10)
            ten_keywords_link=""
            count = 0
            for w in keywords_list:
                count = count+1
                if count<10:
                    ten_keywords_link += "%s. "%count + w["word"]+ "\n"
                else:
                    ten_keywords_link += "%s. "%count + w["word"]
            ####gender

            negative_comments_number = collection_comments.count({"politician_id": politician_dict[politician_name], "created_time": {"$gte": start, "$lte": end}, "score": {"$lt": 0}})
            central_comments_number  = collection_comments.count({"politician_id": politician_dict[politician_name], "created_time": {"$gte": start, "$lte": end}, "score": {"$lte": 0.1, "$gte": 0}})
            positive_comments_number = collection_comments.count({"politician_id": politician_dict[politician_name], "created_time": {"$gte": start, "$lte": end}, "score": {"$gt": 0.1}})
            total_comments_number    = negative_comments_number+central_comments_number+positive_comments_number
            
            positive_comments_persent = positive_comments_number/total_comments_number
            central_comments_persent  = central_comments_number/total_comments_number
            negative_comments_persent = negative_comments_number/total_comments_number
            
            data_list.append((hot,
                              ten_keywords_link,
                              politician_name,
                              round(positive_comments_persent,2),
                              round(negative_comments_persent,2),
                              round(central_comments_persent,2)
                              ))
    return(data_list)

In [55]:
def kmt():
    kmt_data = []
    for politician_name in kmt_politician_dict:
        ###hot
        hot = collection_comments.count({"politician_id":politician_dict[politician_name], 
                                             "created_time": {"$gte": start, "$lte": end}})
        ###gender
        negative_comments_number = collection_comments.count({"politician_id": kmt_politician_dict[politician_name], "created_time": {"$gte": start, "$lte": end}, "score": {"$lt": 0}})
        central_comments_number  = collection_comments.count({"politician_id": kmt_politician_dict[politician_name], "created_time": {"$gte": start, "$lte": end}, "score": {"$lte": 0.1, "$gte": 0}})
        positive_comments_number = collection_comments.count({"politician_id": kmt_politician_dict[politician_name], "created_time": {"$gte": start, "$lte": end}, "score": {"$gt": 0.1}})
        total_comments_number    = negative_comments_number+central_comments_number+positive_comments_number

        positive_comments_persent = round(positive_comments_number/total_comments_number,2)
        central_comments_persent  = round(central_comments_number/total_comments_number,2)
        negative_comments_persent = round(negative_comments_number/total_comments_number,2)
        ###ten_keywords
        keywords_list = collection_keywords.find({"politician_id":kmt_politician_dict[politician_name], 
                                                  "created_time": {"$gte": start, "$lte": end}},
                                                 {"_id":0,"word":1,"link":1}).sort([("new_tfidf", -1)]).limit(10)
        ten_keywords_link=""
        count = 0
        for w in keywords_list:
            count = count+1
            if count<10:
                ten_keywords_link += "%s. "%count + w["word"]+ "\n"
            else:
                ten_keywords_link += "%s. "%count + w["word"]
        kmt_data.append((politician_name,
                         hot,  
                         positive_comments_persent, 
                         central_comments_persent, 
                         negative_comments_persent,
                         ten_keywords_link,
                         kmt_politician_dict[politician_name]))
    return(kmt_data)

In [70]:
kmt_carousel_template_message = TemplateSendMessage(
            alt_text='Carousel template',
            template=CarouselTemplate(
                columns=[
                    CarouselColumn(
                        thumbnail_image_url='https://4b578800.ap.ngrok.io/images/{}'.format(kmt()[0][6]),
                        title ='{}'.format(kmt()[0][0]),
                        text='2017/10 ~ 2017/12 網路聲量\n欲知更多資訊\n請點擊更多 ',
                        actions=[
                            PostbackTemplateAction(
                                label='臉書粉絲回應',
                                data="{} 則回應數".format(kmt()[0][1])
                            ),
                            PostbackTemplateAction(
                                label='情緒正負向',
                                data="正向:{} \n中立:{} \n負向:{}".format(kmt()[0][2],kmt()[0][3],kmt()[0][4]) 
                            ),
#                             PostbackTemplateAction(
#                             label='十大關鍵字',
#                             data="{}".format(kmt[0][5])
#                             ),
                            URITemplateAction(
                                label='更多',
                                uri='https://www.youtube.com/watch?v=uauzw00I0wY'
                            )
               
                        ]
                    ),
                    CarouselColumn(
                        thumbnail_image_url='https://4b578800.ap.ngrok.io/images/{}'.format(kmt()[1][6]),
                        title ='{}'.format(kmt()[1][0]),
                        text='2017/10 ~ 2017/12 網路聲量\n欲知更多資訊\n請點擊更多 ',
                        actions=[
                            PostbackTemplateAction(
                            label='臉書粉絲回應',
                            data="{} 則回應數".format(kmt()[1][1])
                            ),
                            PostbackTemplateAction(
                            label='情緒正負向',
                            data="正向:{} \n中立:{} \n負向:{}".format(kmt()[1][2],kmt()[1][3],kmt()[1][4]) 
                            ),
#                             PostbackTemplateAction(
#                             label='十大關鍵字',
#                             data="{}".format(kmt[1][5])
#                             ),
                            URITemplateAction(
                            label='更多',
                            uri='https://www.youtube.com/watch?v=uauzw00I0wY'
                            )
               
                        ]
                    ),
                    CarouselColumn(
                        thumbnail_image_url='https://4b578800.ap.ngrok.io/images/{}'.format(kmt()[2][6]),
                        title ='{}'.format(kmt()[2][0]),
                        text='2017/10 ~ 2017/12 網路聲量\n欲知更多資訊\n請點擊更多 ',
                        actions=[
                            PostbackTemplateAction(
                            label='臉書粉絲回應',
                            data="{} 則回應數".format(kmt()[2][1])
                            ),
                            PostbackTemplateAction(
                            label='情緒正負向',
                            data="正向:{} \n中立:{} \n負向:{}".format(kmt()[2][2],kmt()[2][3],kmt()[2][4]) 
                            ),
#                             PostbackTemplateAction(
#                             label='十大關鍵字',
#                             data="{}".format(kmt()[2][5])
#                             ),
                            URITemplateAction(
                            label='更多',
                            uri='https://www.youtube.com/watch?v=uauzw00I0wY'
                            )
               
                        ]
                    ),
                    CarouselColumn(
                        thumbnail_image_url='https://4b578800.ap.ngrok.io/images/{}'.format(kmt()[3][6]),
                        title ='{}'.format(kmt()[3][0]),
                        text='2017/10 ~ 2017/12 網路聲量\n欲知更多資訊\n請點擊更多 ',
                        actions=[
                            PostbackTemplateAction(
                            label='臉書粉絲回應',
                            data="{} 則回應數".format(kmt()[3][1])
                            ),
                            PostbackTemplateAction(
                            label='情緒正負向',
                            data="正向:{} \n中立:{} \n負向:{}".format(kmt()[3][2],kmt()[3][3],kmt()[3][4]) 
                            ),
#                             PostbackTemplateAction(
#                             label='十大關鍵字',
#                             data="{}".format(kmt()[3][5])
#                             ),
                            URITemplateAction(
                            label='更多',
                            uri='https://www.youtube.com/watch?v=uauzw00I0wY'
                            )
               
                        ]
                    ),
                    CarouselColumn(
                        thumbnail_image_url='https://4b578800.ap.ngrok.io/images/{}'.format(kmt()[4][6]),
                        title ='{}'.format(kmt()[4][0]),
                        text='2017/10 ~ 2017/12 網路聲量\n欲知更多資訊\n請點擊更多 ',
                        actions=[
                            PostbackTemplateAction(
                            label='臉書粉絲回應',
                            data="{} 則回應數".format(kmt()[4][1])
                            ),
                            PostbackTemplateAction(
                            label='情緒正負向',
                            data="正向:{} \n中立:{} \n負向:{}".format(kmt()[4][2],kmt()[4][3],kmt()[4][4]) 
                            ),
#                             PostbackTemplateAction(
#                             label='十大關鍵字',
#                             data="{}".format(kmt[0][5])
#                             ),
                            URITemplateAction(
                            label='更多',
                            uri='https://www.youtube.com/watch?v=uauzw00I0wY'
                            )
               
                        ]
                    )
                ]))

In [75]:
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/" )

# line_bot_api 用來處理消息
line_bot_api = LineBotApi('81hQGdpPXjmpCwm9EyR84b262JW3bBUfECQj0nEWs0FyWZ3WWSBXxqtvuJ98N5VcyQCvHsWz1xkBLa07rLVAPF9weQxL4wP5YJB3xxMTAw1h/G2rjKmnmnKMhJjfMFq9wEUlTCCRqcw8VN8b+x0sqwdB04t89/1O/w1cDnyilFU=')

# 用來接收外部消息
handler = WebhookHandler('fbc80a3e7c8200bcfbe59100a8da5def')

@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body 驗證封包是否來自Line
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'
###############################################################################
   
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if "國民黨" in event.message.text:
        line_bot_api.reply_message(event.reply_token,kmt_carousel_template_message)
    elif "民進黨" in event.message.text:
        line_bot_api.reply_message(event.reply_token,dpp_carousel_template_message)
    elif "時代力量" in event.message.text:
        line_bot_api.reply_message(event.reply_token,power_carousel_template_message)
    elif "無黨籍" in event.message.text:
        line_bot_api.reply_message(event.reply_token,noparty_carousel_template_message)
    
    
    for data_all in data(event.message.text):
        buttons_template_message = \
        TemplateSendMessage(
            alt_text='Buttons template',
            
            template=ButtonsTemplate(
                thumbnail_image_url='https://4b578800.ap.ngrok.io/images/{}'.format(politician_dict[data_all[2]]),
                title='{}'.format(data_all[2]),
                text='2017/10 ~ 2017/12 網路聲量\n欲知更多資訊\n請點擊更多 ',
                
                actions=[
#                     DatetimePickerTemplateAction(
#                     label="datetime picker date",
#                     data="action=sell&itemid=2&mode=date",
#                     mode="date",
            
#                     min="2017-01-01",
#                     max="2018-01-01"
#                     ),
                    PostbackTemplateAction(
                        label='臉書粉絲回應',
                        data="{} 則回應數".format(data_all[0])
                    ),
                    PostbackTemplateAction(
                        label='情緒正負向',
                        data="正向:{} \n中立:{} \n負向:{}".format( data_all[3], data_all[4], data_all[5] ) 
                    ),
                    PostbackTemplateAction(
                        label='十大關鍵字',
                        data="{}\n\n更多十大關鍵字資訊  請參閱\nhttps://www.youtube.com/watch?v=uauzw00I0wY".format(data_all[1])
                    ),
                    URITemplateAction(
                        label='更多',
                        uri='https://www.youtube.com/watch?v=uauzw00I0wY'
                    )
                    
                ]
            )
        )      
        line_bot_api.push_message(event.source.user_id, buttons_template_message)

    
@handler.add(PostbackEvent)
def handle_postback_message(event):
#     print(event)
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text = event.postback.data))  
#     line_bot_api.push_message(event.source.user_id, TextSendMessage(text="請多多參閱我們的網站\nhttps://dailyview.tw/"))
        
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.1 - - [12/Jan/2018 03:24:15] "POST / HTTP/1.1" 200 -
172.17.0.1 - - [12/Jan/2018 03:24:24] "POST / HTTP/1.1" 200 -
172.17.0.1 - - [12/Jan/2018 03:24:27] "POST / HTTP/1.1" 200 -
172.17.0.1 - - [12/Jan/2018 03:24:28] "POST / HTTP/1.1" 200 -
172.17.0.1 - - [12/Jan/2018 03:24:30] "POST / HTTP/1.1" 200 -
